In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats as stat

In [ ]:
cpy = pd.read_csv('./input/Company.csv')
cpy_tweet = pd.read_csv('./input/Company_Tweet.csv')
tweet = pd.read_csv('./input/Tweet.csv')

In [ ]:
cpy

In [ ]:
cpy_tweet

In [ ]:
tweet

In [ ]:
tweets = pd.merge(tweet,cpy_tweet,on='tweet_id',how='inner')
tweets.head()

In [ ]:
tweets['post_date'] = pd.to_datetime(tweets['post_date'], unit='s')

In [ ]:
tweets['date'] = pd.to_datetime(tweets['post_date'].apply(lambda date: date.date()))

In [ ]:
tweets['date'].head()

In [ ]:
tweets.info()

In [ ]:
tweets = tweets.drop(['tweet_id'],axis=1)

In [ ]:
tweets.isna().sum()

In [ ]:
tweets.ticker_symbol.value_counts()

In [ ]:
aapl = tweets[tweets['ticker_symbol'] == 'AAPL']
tsla = tweets[tweets['ticker_symbol'] == 'TSLA']
amzn = tweets[tweets['ticker_symbol'] == 'AMZN']
msft = tweets[tweets['ticker_symbol'] == 'MSFT']
goog = tweets[tweets['ticker_symbol'] == 'GOOG']
googl = tweets[tweets['ticker_symbol'] == 'GOOGL'] ## With stockholders voting rights

In [ ]:
import yfinance as yf

In [ ]:
tsla_stock = yf.Ticker('TSLA')

In [ ]:
start = min(tsla['date'])
end = max(tsla['date'])

tsla_stock = tsla_stock.history(start=start.date(), end=end.date())

In [ ]:
!pip install afinn

In [ ]:
score= []
for date, group in tsla.groupby("date"):
    aa = group['body'].apply(lambda tweet: afinn.score(tweet)).mean()
    score.append([date,aa])
print(len(score))

In [ ]:
tweets_vol = tsla.groupby('date').size()
print(tweets_vol)

In [ ]:
st = pd.DataFrame(tsla_stock)
vol = pd.DataFrame(tweets_vol)
sc = pd.DataFrame(score)

In [ ]:
res = pd.DataFrame(columns=('date', 'close', 'tweet_vol', 'ts_polarity'))
for row in st.iterrows():
    date = row[0]
    close = row[1]['Close']
    tweet_vol = 'NaN'
    ts_polarity = 'NaN'
    for i in vol.iterrows():
        if i[0]==date:
            tweet_vol = i[1][0]
    for j in sc.iterrows():
        if j[1][0] == date:
            ts_polarity = j[1][1]      
    res = res.append([{'date':date, 'close':close, 'tweet_vol':tweet_vol, 'ts_polarity':ts_polarity}])

In [ ]:
print(res.head())

In [ ]:
res.to_csv('./tesla.csv')